In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = 'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=150&mileage_max=100000&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=55423'
page = requests.get(url)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
columns = ['Year','Make','Model','Used/New','Price','ConsumerRating','ConsumerReviews','SellerType','SellerName','SellerRating','SellerReviews',
           'StreetName','State','Zipcode','DealType','ComfortRating','InteriorDesignRating','PerformanceRating','ValueForMoneyRating',
           'ExteriorStylingRating','ReliabilityRating','ExteriorColor','InteriorColor','Drivetrain','MinMPG','MaxMPG','FuelType',
           'Transmission','Engine','VIN','Stock#','Mileage']   

df= pd.DataFrame(columns=columns)

In [5]:
for i in range(12):
    
    postings = soup.find_all('a', class_ = 'vehicle-card-link js-gallery-click-link')
    current_page = soup
    for post in postings:
        try:
            link = 'http://cars.com' + post.get('href')
            info_page = requests.get(link)
            soup = BeautifulSoup(info_page.text, 'lxml')
            
            ###DELETE THIS WHEN YOURE DONE
            #test_link = 'https://www.cars.com/vehicledetail/1d26a033-1886-4e9c-80cb-fb4f15ff4d51/'
            #info_page = requests.get(test_link)
            #soup = BeautifulSoup(info_page.text, 'lxml')
            
            
            #Data we want
            title_info = soup.find('h1', class_ = 'listing-title').text.split()
            year = title_info[0]
            make = title_info[1]
            model = ' '.join(title_info[2:])
            used_or_new = soup.find('p', class_ = 'new-used').text
            price = soup.find('span', class_ = 'primary-price').text
            consumer_rating = float(soup.find('section', class_= 'sds-page-section vehicle-reviews').find('span', class_='sds-rating__count').text)
            consumer_reviews = int(soup.find('section',class_= 'sds-page-section vehicle-reviews').find('a',class_= 'sds-rating__link sds-button-link').text[1:].split()[0])
            
            #Seller Details
            #Dealer
            try:
                seller_name = soup.find('h3', class_ = 'sds-heading--5 heading seller-name').text
                seller_rating = float(soup.find('section',class_ = 'sds-page-section seller-info').find('span', class_ = 'sds-rating__count').text)
                seller_reviews = soup.find('a', class_ = 'sds-rating__link sds-button-link').text.split()[0][1:]
                seller_type = 'Dealer'
                if ',' in seller_reviews:
                    seller_reviews = int(''.join(seller_reviews.split(',')))
                else:
                    seller_reviews =  int(seller_reviews)
                address_pieces = soup.find('div', class_ = 'dealer-address').text.split(',')
                street_name = address_pieces[0]
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
                
            #Private seller    
            except:
                seller_type = 'Private'
                pieces = soup.find('div', class_ = 'seller-address').text.split('from')
                seller_name = pieces[0].strip()
                address_pieces = pieces[1].split(',')
                street_name = address_pieces[0].strip()
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
    
            
            #Deal type
            deal_type = 'NA'
            first_badge = soup.find('span', class_ = 'sds-badge__label').text
            if 'deal' in first_badge.casefold():
                deal_type = first_badge.split()[0]
            
            #Rating breakdown
            rb = soup.find_all('div', class_ = 'review-breakdown')[0].find_all('span', class_ = 'sds-definition-list__value')
            
            comfort = float(rb[0].text)
            interior_design = float(rb[1].text)
            performance = float(rb[2].text) 
            value_for_money = float(rb[3].text)
            exterior_styling = float(rb[4].text)
            reliability = float(rb[5].text)
            
            #Basic Description Table
            bd = soup.find_all('div', class_ = 'basics-content-wrapper')[0].find_all('dd')
            
            exterior_color = bd[0].text.strip()
            interior_color = bd[1].text.strip()
            drive_train = bd[2].text.strip()
            min_mpg = int(bd[3].text.split('\n')[2].split('–')[0])
            max_mpg = int(bd[3].text.split('\n')[2].split('–')[1])
            fuel_type = bd[4].text.strip()
            transmission = bd[5].text.strip()
            engine = bd[6].text.strip()
            vin = bd[7].text.strip()
            stock_number = bd[8].text.strip()
            mileage = int(''.join(bd[9].text.split()[0].split(',')))
            
            
    
            
            
            
            new_row = pd.DataFrame([[year,make,model,used_or_new,price,consumer_rating,consumer_reviews,seller_type,seller_name,seller_rating,seller_reviews,
                                   street_name,state,zipcode,deal_type,comfort,interior_design,performance,value_for_money,exterior_styling,reliability,
                                   exterior_color,interior_color,drive_train,min_mpg,max_mpg,fuel_type,transmission,engine,vin,stock_number,mileage]],
                                  columns=columns)
            
            #Generate row in dataframe based on scraped information
            df = pd.concat([df,new_row])
        except:
            pass
    
    current_page    
    current_page.find('a', {'aria-label':'Next page'}).get('href')
    next_page = 'http://cars.com' + current_page.find('a', {'aria-label':'Next page'}).get('href') #Get link to next page
        
    url = next_page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    print(i)

#file = 'C:/Users/leave/OneDrive/Documents/Scraped_Datasets/Cars_Data/cars_raw.csv'
file = 'C:/Users/John/Desktop/BOOTCAMP/Car_Search_Project'

#df.to_csv(file, index= False)
df.to_excel("cars_raw.xlsx") 

0
1
2
3
4
5
6
7
8
9
10
11


In [6]:
df

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock#,Mileage
0,2018,Tesla,Model 3 Long Range,Used,"$56,990",4.6,117,Dealer,Luther INFINITI of Bloomington,4.9,...,Black,All-wheel Drive,120,112,Electric,1-Speed Automatic,Electric,5YJ3E1EB6JF088165,P8180A,56046
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$38,643",4.7,42,Dealer,Audi Richfield,4.1,...,Atlas Beige / Gray,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFY6K2029212,K2029212,49873
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$42,977",4.7,42,Dealer,Audi Richfield,4.1,...,Black,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFYXK2132553,K2132553,21978
0,2019,BMW,X5 xDrive40i,Used,"$54,190",4.7,55,Dealer,Motorwerks BMW,4.7,...,Amarone,All-wheel Drive,20,26,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UXCR6C50KLL10954,BN09189T,32415
0,2019,Mercedes-Benz,C-Class C 300 4MATIC,Used,"$38,742",4.8,40,Dealer,Feldmann Imports,4.8,...,Black,All-wheel Drive,22,33,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,55SWF8EB0KU315031,B2752A,13041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2020,Chevrolet,Equinox 1LT,Chevrolet Certified,"$25,990",4.7,277,Dealer,Marthaler Chevrolet of Glenwood,4.5,...,Jet Black,All-wheel Drive,25,30,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,3GNAXUEV9LL239783,220357A,35546
0,2020,Buick,Encore Preferred,Used,"$25,769",4.9,21,Dealer,Kruse Motors,4.7,...,Ebony,Front-wheel Drive,25,30,Gasoline,6-Speed Automatic,1.4L I4 16V MPFI DOHC Turbo,KL4CJASB3LB045094,G0060A,3956
0,2020,Chevrolet,Silverado 1500 LT Trail Boss,Used,"$52,887",4.6,137,Dealer,Lockwood Motors,3.8,...,Gideon,Four-wheel Drive,16,22,Gasoline,10-Speed Automatic,5.3L V8 16V GDI OHV,1GCPYFED7LZ300401,25891A,8768
0,2019,Acura,MDX 3.5L w/Advance Package,Used,"$35,922",4.6,63,Dealer,Lockwood Motors,3.8,...,–,All-wheel Drive,19,26,Gasoline,9-Speed Automatic,3.5L V6 24V GDI SOHC,5J8YD4H81KL013674,26050A,81420
